# Aim
Generate 110 20 node graphs where each parity has 10 diff node degrees

Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

#################
# My notebook issues
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
sys.path.append('home/egupta/.local/bin')

##############

import os
import numpy as np
import scipy.stats as ss
import networkx as nx
import csv 
import numpy as np
import seaborn as sns
from scipy.optimize import curve_fit
import scipy
import matplotlib
from matplotlib import pyplot as plt

%matplotlib inline


##qtensor imports
import qtensor
from qtensor import QAOA_energy
from qtensor import parameter_optimization as popt
import torch

2021-09-19 10:48:28,213- WARNING•	QuickBB solver is unavailable
2021-09-19 10:48:28,216- WARNING•	Tamaki solver is unavailable: No path /home/egupta/.local/lib/python3.7/site-packages/thirdparty/tamaki_treewidth


#### Important Functions

In [2]:
def get_qaoa_params(graph, edge=None, gamma=[0.5], beta=[0.5], steps=50):
    loss_history, params_history = popt.qaoa_maxcut_torch(graph, gamma, beta,
                                                          ordering_algo='greedy',
                                                          Opt=torch.optim.RMSprop,
                                                          edge=edge,
                                                          opt_kwargs=dict(lr=0.002),
                                                          steps=steps,
                                                          pbar=False)
    best_ix = np.argmin(loss_history)
    return params_history[best_ix], loss_history[best_ix]

def get_edge_contribution(graph, edge, gamma, beta):
    sim = qtensor.QtreeSimulator()
    composer = qtensor.DefaultQAOAComposer(graph, gamma=gamma, beta=beta)
    composer.energy_expectation_lightcone(edge)
    return np.real(sim.simulate_batch(composer.circuit)[0])



def opt_runs(G, seeds = 20, steps = 200):
  """
  Doing multiple optimization runs, each with random initialized param
  Returning results which inlclude the optimal param and corresponding energies
  """

  #first do 20 optimization runs 
  s = 0
  g_results = []

  while s<seeds:
    print('seed #' + str(s))
    #initialize param
    gamma = [np.random.rand()]
    beta = [np.random.rand() ]
    initial_energy = QAOA_energy(G,gamma =  gamma, beta = beta)

    #optimization
    params, loss = get_qaoa_params(G, gamma=gamma, beta=beta,edge = None, steps=steps)
    #print('Finished get_qaoa_params')
    gamma_opt, beta_opt = params[0], params[1]
    opt_energy = QAOA_energy(G, gamma = gamma_opt, beta = beta_opt)

    result = np.asarray([float(gamma_opt), float(beta_opt), float(opt_energy), float(initial_energy)])
    g_results.append(result)
    s+=1

  
  return g_results

#### Import Old Graphs

In [3]:
file3 = open("110_20_node_graphs.txt")
mat_list = np.loadtxt(file3).reshape(110,20,20)

def mats_to_graphs(mat_list):
  """
  Given list of adjaceny matrix repr of graphs, converts to actual graphs
  """
  g_list = []
  for mat in mat_list:
    arr = np.array(mat)
    G = nx.convert_matrix.from_numpy_matrix(arr)
    g_list.append(G)
  return g_list

g_list = mats_to_graphs(mat_list)

In [6]:
file4 = open("110_20_node_graphs_data.txt")
g_data = np.loadtxt(file4).reshape(110,20,4)

Import new graphs

In [7]:
file5 = open("New20NodeRandomGraphs.txt")
mat_list = np.loadtxt(file5).reshape(88,20,20)

def mats_to_graphs(mat_list):
  """
  Given list of adjaceny matrix repr of graphs, converts to actual graphs
  """
  g_list = []
  for mat in mat_list:
    arr = np.array(mat)
    G = nx.convert_matrix.from_numpy_matrix(arr)
    g_list.append(G)
  return g_list

new_g_list = mats_to_graphs(mat_list)

In [13]:
new_g_data = []
for i in range(1,7,1):
    file = open("New20NodeRandomGraphsData"+ str(i)+".txt")
    if i<6:
        new_data = np.loadtxt(file).reshape(16,20,4)
    else:
        new_data = np.loadtxt(file).reshape(8,20,4)
    for g_results in new_data:
        new_g_data.append(g_results.tolist())
    

In [15]:
new_g_data

[[[0.5873778462409973,
   0.6170293092727661,
   19.475403759067436,
   13.945312012105074],
  [0.08719003945589066,
   0.38246774673461914,
   19.47529649235906,
   16.540085238297777],
  [0.4125794768333435,
   0.3829449713230133,
   19.47540311530302,
   16.847725041360064],
  [0.08737771958112717,
   0.8829752802848816,
   19.47540365965874,
   17.969007371917318],
  [0.08737781643867493,
   0.38296830654144287,
   19.475403804179518,
   15.075221126492952],
  [0.9126221537590027,
   0.6170356273651123,
   19.4754038688613,
   11.380797684083168],
  [0.9126220941543579,
   1.117053508758545,
   19.475404009767818,
   13.387032217935145],
  [0.9125975966453552,
   0.61705482006073,
   19.475403715324852,
   16.79040888116407],
  [0.9126217365264893,
   0.6170786023139954,
   19.47540378426376,
   10.117954964289714],
  [0.41262176632881165,
   0.3829076886177063,
   19.475403452306434,
   16.930560640211553],
  [0.5873783230781555,
   0.6170935034751892,
   19.475403416461646,
   13

# Sort Graphs

In [18]:
newer_g_list = []
newer_g_data = []
ind = []
deg_seqs = []

for i in range(len(g_list)):
    G = g_list[i]
    Gdata = g_data[i]
    degree_sequence = [d for n, d in G.degree()]
    if degree_sequence not in deg_seqs:
        deg_seqs.append(degree_sequence)
        newer_g_list.append(G)
        newer_g_data.append(Gdata)
        ind.append(i)
        
for i in range(len(new_g_list)):
    G = new_g_list[i]
    Gdata = new_g_data[i]
    degree_sequence = [d for n, d in G.degree()]
    if degree_sequence not in deg_seqs:
        deg_seqs.append(degree_sequence)
        newer_g_list.append(G)
        newer_g_data.append(Gdata)
        ind.append(i)

In [19]:
ind

[0,
 2,
 4,
 6,
 8,
 10,
 12,
 14,
 16,
 18,
 20,
 55,
 60,
 65,
 70,
 75,
 80,
 85,
 90,
 95,
 100,
 105,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87]

In [20]:
len(ind)

110

#### Storing these graphs

In [23]:

def graphs_to_matrix(g_list):
  """
  Converts graphs to matrices
  """
  mat_list = []
  for g in g_list:
    mat = nx.linalg.graphmatrix.adj_matrix(g, weight = None)
    mat = scipy.sparse.csr_matrix.toarray(mat)
    mat_list.append(np.copy(mat))
  return mat_list

In [24]:
new_mat_list = graphs_to_matrix(newer_g_list)

Storing in textfile

In [25]:
a_file = open("New110Distinct20NodeRandomGraphs.txt", "w")
for mat in new_mat_list:
    for row in mat:
        np.savetxt(a_file, row)
a_file.close()

In [22]:
b_file = open("New110Distinct20NodeRandomGraphsData.txt", "w")
for g_res in newer_g_data:
    for res in g_res:
        np.savetxt(b_file, res)
b_file.close()